In [551]:
import re
import pandas as pd
import numpy as np

In [552]:
import PyPDF2
import tabula
from tabula import read_pdf

import textract

ModuleNotFoundError: No module named 'textract'

In [ ]:
OPTIONS: Tabula, PyPDF2, 

In [303]:
ls

data/                             test.ipynb
pdf_reader.ipynb                  valueset_gsheet-checkpoint.ipynb


Errors with Tabula

In [311]:
pdf = read_pdf('data/site-histology.pdf','rb')

In [312]:
#df = tabula.read_pdf("data/site-histology.pdf", encoding='utf-8', spreadsheet=True, pages='1-6041')

In [313]:
type(pdf)

NoneType

In [314]:
pdfFileObj = open('data/site-histology.pdf', 'rb')

In [315]:
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

In [316]:
pdfReader.numPages

380

From pyPDF2 documentation:

extractText()

Locate all text drawing commands, in the order they are provided in the content stream, and extract the text. This works well for some PDF files, but poorly for others, depending on the generator used. This will be refined in the future. Do not rely on the order of text coming out of this function, as it will change if this function is made more sophisticated.

Returns:	a unicode string object.

In [403]:
def get_page(page):
    pageObj = pdfReader.getPage(page)
    return pageObj.extractText()

In [516]:

def get_contents(page):
    pageObj = pdfReader.getPage(page)
    return pageObj.getContents().getData()

In [517]:
text = get_page(3)

In [518]:
contents = get_contents(2)

In [521]:
cont.decode("utf-8")[:1000]

'q 0.48 w 1 J 1 j 71.52 71.52 649.44 487.2 re W* n 1 0 0 1 72 72 cm q /GS0 gs 1.000 1.000 1.000 rg 1.000 1.000 1.000 RG 0.48 w 1 J 1 j 0 0 648 485.76 re B* 0.000 0.000 0.000 rg 0.000 0.000 0.000 RG 0.48 w 1 J 1 j 1.000 1.000 1.000 rg 1.000 1.000 1.000 RG 0.48 w 1 J 1 j 0.000 0.000 0.000 rg 0.000 0.000 0.000 RG 0.48 w 1 J 1 j BT /TT3 11.09 Tf 0 475.68 Td (ICD-0-3 SEER SITE/HISTOLOGY VALIDATION LIST     March 23, 2018                                                                                          3)Tj ET 0 473.76 m 621.6 473.76 l S BT /TT3 8.19 Tf 0 454.08 Td (LIP C000-C006,C008-C009)Tj ET BT /TT2 8.19 Tf 25.92 444.96 Td (LYMPHOEPITHELIAL CARCINOMA)Tj ET BT /TT2 8.19 Tf 317.76 444.96 Td (808)Tj ET BT /TT2 8.19 Tf 350.4 444.96 Td (8081/2)Tj ET BT /TT2 8.19 Tf 389.28 444.96 Td (Bowen disease)Tj ET BT /TT2 8.19 Tf 350.4 435.36 Td (8082/3)Tj ET BT /TT2 8.19 Tf 389.28 435.36 Td (Lymphoepithelial carcinoma)Tj ET BT /TT2 8.19 Tf 350.4 425.76 Td (8083/3)Tj ET BT /TT2 8.19 Tf 389.28 425.

In [522]:
target_df = pd.DataFrame(data=None, columns=['Site Recode','Site Description','Histo','Histology Description','Histology/Behavior','Histology/Behavior Description'])

In [523]:
target_df

,Site Recode,Site Description,Histo,Histology Description,Histology/Behavior,Histology/Behavior Description


for page in range(pdfReader.numPages):
    look for site recode

## histology/behavior
### status: very good


In [524]:
for m in re.finditer(r'([A-Z]{0})([0-9]{4})/([0-9]{1})([A-z]?)', text):
    print('%02d-%02d: %s' % (m.start(), m.end(), m.group(0)[:-1]))

206-213: 8940/3
239-246: 8941/3
294-301: 9140/3
346-353: 9699/3
411-418: 9823/3


## Site Recode:
### status:

In [525]:
#([0-9])(?=[A-Z])
#([A-Z])([0-9]).+(?=[0-9][A-Z])
#([A-Z])([0-9]).+(?=[0-9][A-Z])?
#(C[0-9]{3})
#(?<=:).+?(?=/)
for m in re.finditer(r'(?<=(C([0-9]{3}))).{9}', text):
    print('%02d-%02d: %s' % (m.start(), m.end(), m.group(0)))

161-170: -C006,C00
171-180: -C009MIXE


## Histology description:
### Status: Okay

In [591]:
rand_page = int(np.random.uniform(380))
print(rand_page)

140


In [592]:
text = get_page(rand_page)
for m in re.finditer(r'([A-Z,)])+?(?=( C([0-9]{3})))', text):
    print('%02d-%02d: %s page %d' % (m.start(), m.end(), m.group(0), rand_page))

155-161: LARYNX page 140


In [536]:
text

'ICD-0-3 SEER SITE/HISTOLOGY VALIDATION LIST     March 23, 2018                                                                                          234CERVIXUTERI C530-C531,C538-C539CARCINOSARCOMA, NOS8988980/3Carcinosarcoma, NOS8981/3Carcinosarcoma, embryonal type8982/3Malignant myoepitheliomaMESENCHYMOMA, MALIGNANT8998990/3Mesenchymoma, malignant8991/3Embryonal sarcomaMESONEPHROMA, MALIGNANT9119110/3Mesonephroma, malignantMALIGNANT LYMPHOMA, NOS9599590/3Malignant lymphoma, NOS9591/3Malignant lymphoma, non-Hodgkin9596/3Composite Hodgkin and non-Hodgkin lymphomaHODGKIN LYMPHOMA9659650/3Hodgkin lymphoma, NOS9651/3Hodgkin lymphoma, lymphocyte-rich9652/3Hodgkin lymphoma, mixed cellularity, NOS9653/3Hodgkin lymphoma, lymphocytic deplet., NOS9654/3Hodgkin lymph., lymphocyt. deplet., diffuse fibrosis9655/3Hodgkin lymphoma, lymphocyt. deplet., reticular9659/3Hodgkin lymph., nodular lymphocyte predom.HODGKIN LYMPHOMA, NOD. SCLER.9669661/3Hodgkin granuloma [obs]9662/3Hodgkin sarcoma [obs]9

In [325]:
for m in re.finditer(r'([0-9]{4})/([0-9]{1})', text):
    pd
    print('%02d-%02d: %s' % (m.start(), m.end(), m.group(0)))

187-193: 8000/3
212-218: 8001/3
240-246: 8002/3
278-284: 8003/3
316-322: 8004/3
356-362: 8005/3
411-417: 8010/2
439-445: 8010/3
459-465: 8011/3
487-493: 8012/3
518-524: 8013/3
559-565: 8014/3
609-615: 8015/3
662-668: 8020/3
705-711: 8021/3
742-748: 8022/3
802-808: 8030/3
845-851: 8031/3
871-877: 8032/3
899-905: 8033/3
932-938: 8034/3
962-968: 8035/3
1037-1043: 8050/2
1070-1076: 8050/3
1100-1106: 8051/3
1130-1136: 8052/2
1183-1189: 8052/3
1253-1259: 8070/2
1295-1301: 8070/3
1329-1335: 8071/3
1372-1378: 8072/3
1416-1422: 8073/3
1460-1466: 8074/3
1498-1504: 8075/3
1536-1542: 8076/2
1594-1600: 8076/3
1634-1640: 8078/3


4 int + / + int

In [196]:
#re = \d{4}'\'\d{1}
x = page

AttributeError: 'list' object has no attribute 'split'

In [593]:
page = get_page(1)

In [595]:
get_page(5).split(' ')
#.split('/')

['ICD-0-3',
 'SEER',
 'SITE/HISTOLOGY',
 'VALIDATION',
 'LIST',
 '',
 '',
 '',
 '',
 'March',
 '23,',
 '2018',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '6BASEOFTONGUE',
 'C019LYMPHOEPITHELIAL',
 'CARCINOMA8088081/2Bowen',
 'disease8082/3Lymphoepithelial',
 'carcinoma8083/3Basaloid',
 'squamous',
 'cell',
 'carcinoma8084/3Squamous',
 'cell',
 'carcinoma,',
 'clear',
 'cell',
 'type8085/3Squamous',
 'cell',
 'carcinoma,',
 'HPV-positive8086/3Squamous',
 'cell',
 'carcinoma,',
 'HPV-negativeADENOCARCINOMA,',
 'NOS8148140/2Adenocarcinoma',
 'in',
 'situ8140/3Adenocarcinoma,',
 'NOS8141/3Scirr

In [207]:
result_string = ''
re.match("d",page)

In [202]:
type(get_page(1))

str

In [91]:
print('this page contains %d characters' % (len(get_page(2))))

this page contains 1613 characters


In [271]:
regex = re.compile(r'([0-9]{2}/)+([0-9]{1})([A-Z])')
regex2 = re.compile(r'/([0-9]{1})')
regex3 = re.compile(r'([0-9]{2}/)([0-9]{1})([A-Z])')

page = regex3.split(get_page(2))
page2 = regex2.split(get_page(1))

In [272]:
for line in page:
    print(line)

ICD-0-3 SEER SITE/HISTOLOGY VALIDATION LIST     March 23, 2018                                                                                          3LIP C000-C006,C008-C009LYMPHOEPITHELIAL CARCINOMA80880
81/
2
B
owen disease80
82/
3
L
ymphoepithelial carcinoma80
83/
3
B
asaloid squamous cell carcinoma80
84/
3
S
quamous cell carcinoma, clear cell typeADENOCARCINOMA, NOS81481
40/
2
A
denocarcinoma in situ81
40/
3
A
denocarcinoma, NOS81
41/
3
S
cirrhous adenocarcinoma81
43/
3
S
uperficial spreading adenocarcinoma81
47/
3
B
asal cell adenocarcinomaADENOID CYSTIC & CRIBRIFORM CA.82082
00/
3
A
denoid cystic carcinoma82
01/
2
C
ribriform carcinoma in situ82
01/
3
C
ribriform carcinomaBRONCHIOLO-ALVEOLAR ADENOCA.82582
55/
3
A
denocarcinoma with mixed subtypesPAPILLARY ADENOCARCINOMA, NOS82682
60/
3
P
apillary adenocarcinoma, NOS82
61/
2
A
denocarcinoma in situ in villous adenoma82
61/
3
A
denocarcinoma in villous adenoma82
62/
3
V
illous adenocarcinoma82
63/
2
A
denocarcinoma in situ in tu

In [254]:
for line in page2:
    print(line)

ICD-0-3 SEER SITE/HISTOLOGY VALIDATION LIST     March 23, 2018                                                                                          2LIP C000-C006,C008-C009NEOPLASM8008000
3
Neoplasm, malignant8001
3
Tumor cells, malignant8002
3
Malignant tumor, small cell type8003
3
Malignant tumor, giant cell type8004
3
Malignant tumor, spindle cell type8005
3
Malignant tumor, clear cell typeCARCINOMA, NOS8018010
2
Carcinoma in situ, NOS8010
3
Carcinoma, NOS8011
3
Epithelioma, malignant8012
3
Large cell carcinoma, NOS8013
3
Large cell neuroendocrine carcinoma8014
3
Large cell carcinoma with rhabdoid phenotype8015
3
Glassy cell carcinomaCARCINOMA, UNDIFF., NOS8028020
3
Carcinoma, undifferentiated type, NOS8021
3
Carcinoma, anaplastic type, NOS8022
3
Pleomorphic carcinomaGIANT & SPINDLE CELL CARCINOMA8038030
3
Giant cell and spindle cell carcinoma8031
3
Giant cell carcinoma8032
3
Spindle cell carcinoma8033
3
Pseudosarcomatous carcinoma8034
3
Polygonal cell carcinoma8035
3
Carcinoma 

In [234]:
len(page)

77

In [94]:
print(1)

1
